# BÀI THỰC HÀNH 4: MẠNG NEURAL HỒI QUY

<b>Hướng dẫn nộp bài:</b> Các bạn commit và push code lên github, sử dụng file txt đặt tên theo cú pháp <MSSV>.txt chứa đường link dẫn đến github của bài thực hành và nộp file txt này tên courses.

Bộ dữ liệu sử dụng: [PhoMT](https://drive.google.com/drive/folders/1ksAAeUq2b4u_hiKpyzaEDNzfQDHoB1cI?usp=sharing).

#### Bài 1: Xây dựng kiến trúc Encoder-Decoder gồm 3 lớp LSTM cho module encoder và 3 lớp LSTM cho module decoder, với hidden size là 256, cho bài toán dịch máy từ tiếng Anh sang tiếng Việt. Huấn luyện mô hình này trên bộ dữ liệu PhoMT sử dụng Adam làm phương thức tối ưu tham số. Đánh giá độ hiệu quả của mô hình sử dụng độ đo ROUGE-L.

### Set up and download data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
base_dir = '/content/drive/MyDrive/Uni/Ki-5-Nam-3/DS201/Lab-4'
os.chdir(base_dir)

In [ ]:
%%capture
!pip install gdown

In [ ]:
import gdown
import os

download_dir = base_dir + "/dataset"
os.makedirs(download_dir, exist_ok=True)

train_file_id = '1-eG6FeF-v__rsf77iWurddahXbyjTYh5'
dev_file_id = '1hoTd2hFwjSeFThlPm6YpN0NW5ePXS3Jc'
test_file_id = '1_3L25SH1_jaEfOjpmpgnfMik4N3MxSyn'

gdown.download(id=train_file_id, output=os.path.join(download_dir, 'train.json'), quiet=False)
gdown.download(id=dev_file_id, output=os.path.join(download_dir, 'dev.json'), quiet=False)
gdown.download(id=test_file_id, output=os.path.join(download_dir, 'test.json'), quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1-eG6FeF-v__rsf77iWurddahXbyjTYh5
To: /content/drive/MyDrive/Uni/Ki-5-Nam-3/DS201/Lab-4/dataset/train.json
100%|██████████| 5.68M/5.68M [00:00<00:00, 63.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1hoTd2hFwjSeFThlPm6YpN0NW5ePXS3Jc
To: /content/drive/MyDrive/Uni/Ki-5-Nam-3/DS201/Lab-4/dataset/dev.json
100%|██████████| 594k/594k [00:00<00:00, 65.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1_3L25SH1_jaEfOjpmpgnfMik4N3MxSyn
To: /content/drive/MyDrive/Uni/Ki-5-Nam-3/DS201/Lab-4/dataset/test.json
100%|██████████| 669k/669k [00:00<00:00, 108MB/s]


'/content/drive/MyDrive/Uni/Ki-5-Nam-3/DS201/Lab-4/dataset/test.json'

### Bài 1

In [ ]:
!pip install -r module_bai_1/requirements.txt

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=ccf7207bfa046e4a1b065302cf9cb114eab7e9c3de8ddbd78a2c0d284185bcac
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [ ]:
from module_bai_1.config import Config
from module_bai_1.vocabulary import Vocabulary
from module_bai_1.data_loader import build_vocabularies, create_dataloaders
from module_bai_1.model import Encoder, Decoder, Seq2Seq
from module_bai_1.train import train_model
from module_bai_1.evaluate import evaluate_model, show_translation_examples

In [ ]:
config = Config()
print(config)

Configuration:
  device: cuda
  hidden_size: 256
  num_layers: 3
  embedding_dim: 256
  dropout: 0.3
  batch_size: 32
  learning_rate: 0.0005
  num_epochs: 15
  max_length: 128
  early_stopping_patience: 3
  train_path: dataset/train.json
  dev_path: dataset/dev.json
  test_path: dataset/test.json
  teacher_forcing_ratio: 0.7



In [ ]:
src_vocab, tgt_vocab = build_vocabularies(config)

[INFO] Building vocab...
Source vocabulary size: 17418
Target vocabulary size: 7061


In [ ]:
train_loader, dev_loader, test_loader = create_dataloaders(config, src_vocab, tgt_vocab)
print(f"Train: {len(train_loader.dataset)} samples")
print(f"Dev: {len(dev_loader.dataset)} samples")
print(f"Test: {len(test_loader.dataset)} samples")

Train: 20000 samples
Dev: 2000 samples
Test: 2000 samples


In [ ]:
encoder = Encoder(len(src_vocab), config.embedding_dim, config.hidden_size,
                  config.num_layers, config.dropout)
decoder = Decoder(len(tgt_vocab), config.embedding_dim, config.hidden_size,
                  config.num_layers, config.dropout)
model = Seq2Seq(encoder, decoder, config.device).to(config.device)

In [ ]:
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

Parameters: 11,239,317


In [ ]:
train_losses, val_losses = train_model(model, train_loader, dev_loader, config)

[INFO] Starting training...
Early stopping patience: 3

Epoch 1/15


Evaluating: 100%|██████████| 63/63 [00:09<00:00,  6.93it/s]


Train Loss: 6.2580
Val Loss: 6.4666
Learning Rate: 0.000500
Best model updated (val loss: 6.4666)

Epoch 2/15


Evaluating: 100%|██████████| 63/63 [00:09<00:00,  6.51it/s]


Train Loss: 6.0280
Val Loss: 6.4917
Learning Rate: 0.000500
[INFO] No improvement. Patience: 1/3

Epoch 3/15


Evaluating: 100%|██████████| 63/63 [00:09<00:00,  6.47it/s]


Train Loss: 5.8217
Val Loss: 6.5492
Learning Rate: 0.000500
[INFO] No improvement. Patience: 2/3

Epoch 4/15


Evaluating: 100%|██████████| 63/63 [00:09<00:00,  6.43it/s]

Train Loss: 5.6919
Val Loss: 6.6761
Learning Rate: 0.000250 (reduced from 0.000500)
[INFO] No improvement. Patience: 3/3

[INFO] Early stopping triggered after 4 epochs
[INFO] Best validation loss: 6.4666

Loaded best model with validation loss: 6.4666


In [ ]:
results = evaluate_model(model, test_loader, config, tgt_vocab)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Generating translations...


100%|██████████| 63/63 [00:05<00:00, 12.34it/s]


Evaluation Results:
ROUGE-L     : 0.1886


In [ ]:
show_translation_examples(model, test_loader, config, src_vocab, tgt_vocab, num_examples=5)


Translation Examples:

Example 1:
Source:      brother albert barnett and his wife , sister susan barnett , from the west congregation in tuscaloosa , alabama
Reference:   anh albert barnett và chị susan barnett , thuộc hội thánh west ở tuscaloosa , alabama
Translation: và tôi , tôi tôi có thể , tôi , tôi có thể , tôi , tôi có thể , tôi , tôi có thể , và tôi , tôi có thể , và tôi , tôi có thể , và tôi , tôi có thể , và tôi , tôi có thể , và tôi , tôi có thể , và tôi , tôi có thể , và tôi , tôi có thể .
Indices:     [6, 7, 4, 7, 7, 9, 20, 4, 7, 4]...
--------------------------------------------------------------------------------

Example 2:
Source:      severe storms ripped through parts of the southern and midwestern united states on january 11 and 12 , 2020 .
Reference:   ngày 11 và 12-1-2020 , những cơn bão lớn đã quét qua và phá huỷ nhiều vùng ở miền nam và miền trung hoa kỳ .
Translation: và tôi , tôi tôi có thể , tôi , tôi có thể , tôi , tôi có thể , tôi , tôi có thể , và tôi , 

#### Bài 2: Xây dựng kiến trúc Encoder-Decoder gồm 3 lớp LSTM cho module encoder và 3 lớp LSTM cho module decoder, với hidden size là 256, cho bài toán dịch máy từ tiếng Anh sang tiếng Việt. Module decoder được trang bị kỹ thuật attention theo mô tả của nghiên cứu "[Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473)". Huấn luyện mô hình này trên bộ dữ liệu PhoMT sử dụng Adam làm phương thức tối ưu tham số. Đánh giá độ hiệu quả của mô hình sử dụn độ đo ROUGE-L.

In [3]:
!pip install -r module_bai_2/requirements.txt

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=5e224460cc767246e74d822b993f935a44a5db22469223bcccf1b02448c01997
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [3]:
from module_bai_2.config import Config
from module_bai_2.data_loader import build_vocabularies, create_dataloaders
from module_bai_2.model import Encoder, Decoder, Seq2Seq
from module_bai_2.train import train_model
from module_bai_2.evaluate import evaluate_model, show_translation_examples
import torch, torch.nn as nn

In [4]:
config = Config()
print(config)

Configuration:
  device: cuda
  hidden_size: 256
  num_layers: 3
  embedding_dim: 256
  dropout: 0.3
  batch_size: 32
  learning_rate: 0.0007
  num_epochs: 15
  max_length: 128
  early_stopping_patience: 3
  train_path: dataset/train.json
  dev_path: dataset/dev.json
  test_path: dataset/test.json
  train_sample_size: 20000
  dev_sample_size: 2000
  test_sample_size: 2000
  teacher_forcing_ratio: 0.7



In [5]:
src_vocab, tgt_vocab = build_vocabularies(config)

In [6]:
train_loader, dev_loader, test_loader = create_dataloaders(config, src_vocab, tgt_vocab)

In [7]:
from module_bai_2.main import build_model
model = build_model(config, len(src_vocab), len(tgt_vocab))

In [8]:
train_losses, val_losses = train_model(model, train_loader, dev_loader, config)


Epoch 1/15


Evaluating: 100%|██████████| 63/63 [00:10<00:00,  6.26it/s]


Train Loss: 6.0992
Val Loss: 6.3271
Learning Rate: 0.000700
Best model updated (val loss: 6.3271)

Epoch 2/15


Evaluating: 100%|██████████| 63/63 [00:10<00:00,  6.27it/s]


Train Loss: 5.5400
Val Loss: 6.3190
Learning Rate: 0.000700
Best model updated (val loss: 6.3190)

Epoch 3/15


...

Train Loss: 4.5968
Val Loss: 6.2658
Learning Rate: 0.000700
No improvement. Patience: 1/3

Epoch 8/15


Evaluating: 100%|██████████| 63/63 [00:10<00:00,  6.29it/s]


Train Loss: 4.4940
Val Loss: 6.2516
Learning Rate: 0.000700
No improvement. Patience: 2/3

Epoch 9/15


Evaluating: 100%|██████████| 63/63 [00:10<00:00,  6.25it/s]

Train Loss: 4.4064
Val Loss: 6.3129
Learning Rate: 0.000350 (reduced from 0.000700)
No improvement. Patience: 3/3
Early stopping at epoch 9
Loaded best model with validation loss: 6.2319


In [9]:
evaluate_model(model, test_loader, config, tgt_vocab)


[INFO] Generating translations for ROUGE-L evaluation


100%|██████████| 63/63 [00:07<00:00,  8.20it/s]

ROUGE-L: 0.3436


0.3436026414194692

In [10]:
show_translation_examples(model, test_loader, config, src_vocab, tgt_vocab, num_examples=5)


Example 1
Source:      brother albert barnett and his wife , sister susan barnett , from the west congregation in tuscaloosa , alabama
Reference:   anh albert barnett và chị susan barnett , thuộc hội thánh west ở tuscaloosa , alabama
Translation: cô ấy đã và và và , , , , , , , , , , , , , , , , ,

Example 2
Source:      severe storms ripped through parts of the southern and midwestern united states on january 11 and 12 , 2020 .
Reference:   ngày 11 và 12-1-2020 , những cơn bão lớn đã quét qua và phá huỷ nhiều vùng ở miền nam và miền trung hoa kỳ .
Translation: những công ty của các công ty của các và và và và và và và và và và , , và , và , và , và và .

Example 3
Source:      two days of heavy rain , high winds , and numerous tornadoes caused major damage across multiple states .
Reference:   những trận mưa to và gió lớn trong suốt hai ngày cùng với nhiều cơn lốc xoáy đã gây thiệt hại nặng nề cho nhiều bang .
Translation: một năm trước , , , , , , và , và , và , và các nhà khoa học 

#### Bài 3: Xây dựng kiến trúc Encoder-Decoder gồm 3 lớp LSTM cho module encoder và 3 lớp LSTM cho module decoder, với hidden size là 256, cho bài toán dịch máy từ tiếng Anh sang tiếng Việt. Module decoder được trang bị kỹ thuật attention theo mô tả của nghiên cứu "[Effective Approaches to Attention-based Neural Machine Translation](https://arxiv.org/abs/1508.04025)". Huấn luyện mô hình này trên bộ dữ liệu PhoMT sử dụng Adam làm phương thức tối ưu tham số. Đánh giá độ hiệu quả của mô hình sử dụn độ đo ROUGE-L.

In [3]:
from module_bai_3.config import Config
from module_bai_3.data_loader import build_vocabularies, create_dataloaders
from module_bai_3.model import Encoder, Decoder, Seq2Seq
from module_bai_3.train import train_model
from module_bai_3.evaluate import evaluate_model, show_translation_examples
import torch, torch.nn as nn

In [5]:
!python -m module_bai_3.main


Epoch 1/15
Training: 100% 625/625 [04:49<00:00,  2.16it/s]
Evaluating: 100% 63/63 [00:06<00:00,  9.39it/s]
Train Loss: 6.0938
Val Loss: 6.3829
Learning Rate: 0.000700
Best model updated (val loss: 6.3829)

Epoch 2/15
Training: 100% 625/625 [04:49<00:00,  2.16it/s]
Evaluating: 100% 63/63 [00:06<00:00,  9.71it/s]
Train Loss: 5.7652
Val Loss: 6.2587
Learning Rate: 0.000700
Best model updated (val loss: 6.2587)

Epoch 3/15
Training: 100% 625/625 [04:49<00:00,  2.16it/s]
Evaluating: 100% 63/63 [00:05<00:00, 10.62it/s]
Train Loss: 5.5455
Val Loss: 6.2276
Learning Rate: 0.000700
Best model updated (val loss: 6.2276)

Epoch 4/15
Training: 100% 625/625 [04:50<00:00,  2.15it/s]
Evaluating: 100% 63/63 [00:06<00:00, 10.49it/s]
Train Loss: 5.3487
Val Loss: 6.1355
Learning Rate: 0.000700
Best model updated (val loss: 6.1355)

Epoch 5/15
Training: 100% 625/625 [04:49<00:00,  2.16it/s]
Evaluating: 100% 63/63 [00:05<00:00, 10.63it/s]
Train Loss: 5.1745
Val Loss: 6.0822
Learning Rate: 0.000700
Best mod